In [1]:
import csv
import collections
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt

In [2]:
# This is the grid object, which is used throughout all data preprocessing.
# It represents the city of Austin through a series of grids.
# It thus makes a tractable way to compute distance between grids, ect. 
class Grid():
    def __init__(self, grid_json):
        self.grid = grid_json
        self.min_lat = self.grid["latitude_min"]
        self.min_lon = self.grid["longitude_min"]
        self.max_lat = self.grid["latitude_max"]
        self.max_lon = self.grid["longitude_max"]
        self.latitude_delta = self.grid["latitude_step"]
        self.longitude_delta = self.grid["longitude_step"]
        self.nrows = math.ceil((self.max_lat - self.min_lat) / self.latitude_delta)
        self.ncols = math.ceil((self.max_lon - self.min_lon) / self.longitude_delta)
        self.times = self.grid["time_matrix"]
        self.census_tract_region_map = self.grid["census_tract_region_mapping"]
        self.region_to_tract = collections.defaultdict(list)
        for census_tract in self.census_tract_region_map:
            for region in self.census_tract_region_map[census_tract]:
                self.region_to_tract[region].append(census_tract)
    def map_point_to_region(self, latitude, longitude):
        return math.floor((latitude-self.min_lat)/self.latitude_delta) * self.ncols  + math.floor((longitude-self.min_lon)/self.longitude_delta)
    def get_representative(self, region_num):
        row_num = region_num//self.ncols
        col_num = region_num - row_num*self.ncols
        lat = self.min_lat + row_num * self.latitude_delta + 0.5*self.latitude_delta
        lon = self.min_lon + col_num * self.longitude_delta + 0.5*self.longitude_delta
        return [lon, lat]
    def get_time(self, region1, region2):
        try:
            return self.times[region1][region2]
        except IndexError:
            return -1
    def region_to_census_tract(self, region):
        try:
            return self.region_to_tract[region]
        except KeyError:
            return "0_0"

In [3]:
# Using smaller distance matrix for hopefully faster runtime of Julia code
with open("Input_Data/grid_info_smaller.json", "r") as f:
    grid_json = json.load(f)

In [4]:
g = Grid(grid_json)

numregions = g.nrows * g.ncols

In [20]:
print(len(g.grid['time_matrix']))
g.grid['time_matrix'][0]

200


[300,
 1917.78,
 2057.79,
 2379.26,
 3991.0,
 3471.98,
 3814.12,
 4184.63,
 4468.9,
 4510.44,
 5966.19,
 5235.99,
 5427.24,
 6351.45,
 6579.91,
 2535.71,
 2148.78,
 2705.97,
 3414.89,
 3583.7,
 3744.2,
 3786.81,
 4407.25,
 4362.16,
 4811.0,
 5000.15,
 5028.13,
 5151.51,
 5661.09,
 5929.15,
 2331.03,
 1976.2,
 2691.27,
 3486.61,
 5030.04,
 3886.35,
 4154.82,
 4416.85,
 4562.59,
 4844.96,
 5201.5,
 5371.67,
 5298.25,
 5602.76,
 5986.81,
 3159.6,
 2450.04,
 3265.35,
 3333.37,
 3814.65,
 4195.22,
 4281.9,
 4491.57,
 4844.5,
 4672.04,
 5043.23,
 4896.13,
 5905.5,
 6018.79,
 6622.02,
 2910.67,
 2674.75,
 3150.62,
 3224.91,
 3507.32,
 3584.18,
 4114.03,
 4204.08,
 4486.16,
 4754.54,
 5678.82,
 5850.48,
 5933.11,
 6585.4,
 6937.2,
 3350.79,
 3050.17,
 3425.82,
 4273.43,
 3638.67,
 4080.24,
 4162.1,
 4224.12,
 4648.37,
 5282.88,
 5079.71,
 5441.1,
 6118.78,
 6553.99,
 7400.59,
 4949.22,
 3382.71,
 3979.33,
 3721.74,
 3982.3,
 4451.97,
 5026.4,
 4200.76,
 4757.58,
 5168.61,
 5418.33,
 5644.85,
 